In [14]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [15]:
# CELLULE 2 : Chargement des données
df = pd.read_csv('inner_merge.csv')
df.shape

(8168, 73)

In [16]:
# CELLULE 3 : Suppression des doublons (CRITIQUE)
print(f"Doublons détectés: {df.duplicated().sum()}")
print(f"Doublons sur titre: {df['movie_original_title'].duplicated().sum()}")

# Supprimer les doublons
df = df.drop_duplicates(subset='movie_original_title', keep='first')
df = df.reset_index(drop=True)

df.shape

Doublons détectés: 0
Doublons sur titre: 7306


(862, 73)

In [17]:
# CELLULE 4 : Renommage des colonnes
df = df.rename(columns={
    'movie_original_title': 'title',
    'movie_genres_y': 'genres',
    'movie_overview': 'overview'
})

In [18]:
# CELLULE 5 : Gestion des valeurs manquantes
df['title'] = df['title'].fillna('')
df['genres'] = df['genres'].fillna('')
df['keywords'] = df['keywords'].fillna('')
df['overview'] = df['overview'].fillna('')

In [19]:
# CELLULE 6 : Fonction d'extraction des noms
def extract_names(x):
    if isinstance(x, str):
        try:
            x = ast.literal_eval(x)
        except:
            return []
    if isinstance(x, list):
        return [d.get('name') for d in x if isinstance(d, dict)]
    return []

In [20]:
# CELLULE 7 : Préparation des genres
df['genres_text'] = df['genres'].str.replace(',', ' ')

print(f" genres: {df['genres_text'].iloc[0]}")

 genres: Comedy Drama


In [21]:
# CELLULE 8 : Préparation des keywords
df['keywords_clean'] = df['keywords'].apply(extract_names)
df['keywords_text'] = df['keywords_clean'].apply(lambda lst: " ".join(lst))
print(f"keywords: {df['keywords_text'].iloc[0][:100]}...")

keywords: movie business violinist filmmaking existential crisis...


In [22]:
# CELLULE 9 : Création des dummies de genres
genres_dummies = df['genres_text'].str.get_dummies(sep=' ')
genres_dummies = genres_dummies.add_prefix('Genre_')

print(f"Nombre de genres uniques: {genres_dummies.shape[1]}")

Nombre de genres uniques: 20


In [23]:
# CELLULE 10 : TF-IDF sur les keywords
tfidf = TfidfVectorizer(stop_words='english', max_features=300)
tfidf_matrix = tfidf.fit_transform(df['keywords_text'])

tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(), 
    index=df.index, 
    columns=[f'TFIDF_{col}' for col in tfidf.get_feature_names_out()]
)
print(f"Nombre de features TF-IDF: {tfidf_df.shape[1]}")

Nombre de features TF-IDF: 300


In [24]:
# CELLULE 11 : Construction de la matrice X
X = pd.concat([genres_dummies, tfidf_df], axis=1)
print(f"Matrice X finale: {X.shape}")
print(f"  - Genres: {genres_dummies.shape[1]}")
print(f"  - TF-IDF: {tfidf_df.shape[1]}")

Matrice X finale: (862, 320)
  - Genres: 20
  - TF-IDF: 300


In [25]:
# CELLULE 12 : Entraînement du modèle KNN
model_knn = NearestNeighbors(n_neighbors=6, metric='cosine')
model_knn.fit(X)

,n_neighbors,6
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [26]:
# CELLULE 13 : Fonction de recommandation
def recommend_movies(movie_title, df, model_knn, X, n_recommendations=5):
    # Trouver l'index du film
    try:
        movie_index = df[df['title'].str.lower() == movie_title.lower()].index[0]
    except IndexError:
        return f"Film '{movie_title}' non trouvé"
    
    # Récupérer les features
    movie_features = X.iloc[movie_index].values.reshape(1, -1)
    
    # Trouver les voisins
    distances, indices = model_knn.kneighbors(movie_features, n_neighbors=n_recommendations+1)
    
    # Créer la liste (ignorer le film lui-même à l'index 0)
    recommendations = []
    for i in range(1, len(indices.flatten())):
        rec_idx = indices.flatten()[i]
        recommendations.append(df.iloc[rec_idx]['title'])
    
    return recommendations

In [27]:
# CELLULE 14 : Test des recommandations
movie_title = 'Scarface'

# Vérifier si le film existe
if movie_title in df['title'].values:
    recos = recommend_movies(movie_title, df, model_knn, X)
    
    print(f"\nRecommandations pour '{movie_title}':")
    for i, title in enumerate(recos, 1):
        print(f"{i}. {title}")
else:
    print(f"Film '{movie_title}' non trouvé")
    print("\nExemples de films disponibles:")
    print(df['title'].head(10).tolist())


Recommandations pour 'Scarface':
1. Drugstore Cowboy
2. Jalan Pintas
3. Sweet Sixteen
4. Pulp Fiction
5. Casino


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [28]:
# CELLULE 15 : Fonction de recommandation détaillée
def recommend_detailed(movie_title, df, model_knn, X, n_recommendations=5):
    try:
        movie_index = df[df['title'].str.lower() == movie_title.lower()].index[0]
    except IndexError:
        print(f"Film non trouvé: {movie_title}")
        return
    
    # Info du film original
    print(f"\nFilm original: {df.iloc[movie_index]['title']}")
    print(f"Genres: {df.iloc[movie_index]['genres_text']}")
    
    # Récupérer les features
    movie_features = X.iloc[movie_index].values.reshape(1, -1)
    distances, indices = model_knn.kneighbors(movie_features, n_neighbors=n_recommendations+1)
    
    # Afficher les recommandations
    print(f"\nTop {n_recommendations} recommandations:\n")
    
    for i in range(1, len(indices.flatten())):
        rec_idx = indices.flatten()[i]
        distance = distances.flatten()[i]
        similarity = (1 - distance) * 100
        
        print(f"{i}. {df.iloc[rec_idx]['title']}")
        print(f"   Genres: {df.iloc[rec_idx]['genres_text']}")
        print(f"   Similarité: {similarity:.1f}%\n")

print("Fonction détaillée définie")

Fonction détaillée définie


In [29]:
# CELLULE 16 : Test avec détails
recommend_detailed('Avatar', df, model_knn, X)


Film original: Avatar
Genres: Action Adventure Fantasy

Top 5 recommandations:

1. The Empire Strikes Back
   Genres: Action Adventure Fantasy
   Similarité: 85.2%

2. Return of the Jedi
   Genres: Action Adventure Fantasy
   Similarité: 82.7%

3. Avatar: The Way of Water
   Genres: Action Adventure Fantasy
   Similarité: 82.0%

4. Star Wars: Episode III - Revenge of the Sith
   Genres: Action Adventure Fantasy
   Similarité: 80.4%

5. The Mummy
   Genres: Action Adventure Fantasy
   Similarité: 76.7%



/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [30]:
# CELLULE 17 : Recherche de films
def search_movies(keyword, df):
    matches = df[df['title'].str.contains(keyword, case=False, na=False)]
    
    if len(matches) == 0:
        print(f"Aucun film trouvé avec '{keyword}'")
        return []
    
    print(f"\n{len(matches)} film(s) trouvé(s):")
    for i, (idx, row) in enumerate(matches.head(10).iterrows(), 1):
        print(f"{i}. {row['title']} - {row['genres_text']}")
    
    return matches['title'].tolist()

In [31]:
# CELLULE 18 : Tester la recherche
search_movies('Star', df)


9 film(s) trouvé(s):
1. Stardust Memories - Comedy Drama
2. Star Wars: The Force Awakens - Action Adventure Sci-Fi
3. Star Wars: Episode III - Revenge of the Sith - Action Adventure Fantasy
4. Star Trek - Action Adventure Sci-Fi
5. The Fault in Our Stars - Drama Romance
6. Stargate - Action Adventure Sci-Fi
7. Starship Troopers - Action Adventure Sci-Fi
8. Star Trek Beyond - Action Adventure Sci-Fi
9. A Star Is Born - Drama Music Romance


['Stardust Memories',
 'Star Wars: The Force Awakens',
 'Star Wars: Episode III - Revenge of the Sith',
 'Star Trek',
 'The Fault in Our Stars',
 'Stargate',
 'Starship Troopers',
 'Star Trek Beyond',
 'A Star Is Born']

In [32]:
# CELLULE 19 : Vérification de la qualité
def check_duplicates_in_recommendations(df, model_knn, X, n_tests=10):
    sample_indices = np.random.choice(df.index, min(n_tests, len(df)), replace=False)
    
    for idx in sample_indices:
        movie_title = df.iloc[idx]['title']
        recos = recommend_movies(movie_title, df, model_knn, X)
        
        unique_recos = len(set(recos))
        
        if unique_recos < len(recos):
            print(f"ATTENTION: Doublons détectés pour '{movie_title}'")
            print(f"  Recommandations: {recos}\n")
        else:
            print(f"Uwu: {movie_title} - {unique_recos} films uniques")

check_duplicates_in_recommendations(df, model_knn, X)

Uwu: Lilo & Stitch - 5 films uniques
Uwu: E.T. the Extra-Terrestrial - 5 films uniques
Uwu: Prometheus - 5 films uniques
Uwu: Shrek - 5 films uniques
Uwu: American Sniper - 5 films uniques
Uwu: Caught Stealing - 5 films uniques
Uwu: Sleepers - 5 films uniques
Uwu: Drive - 5 films uniques
Uwu: Interview with the Vampire - 5 films uniques
Uwu: Jane B. par Agnès V. - 5 films uniques


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/opt/homebrew/lib/python3

In [33]:
# CELLULE 20 : Statistiques globales
print(f"\nStatistiques du système:")
print(f"Nombre total de films: {len(df)}")
print(f"Nombre de features: {X.shape[1]}")
print(f"Films avec keywords: {(df['keywords_text'] != '').sum()}")
print(f"Films avec genres: {(df['genres_text'] != '').sum()}")


Statistiques du système:
Nombre total de films: 862
Nombre de features: 320
Films avec keywords: 826
Films avec genres: 862


In [ ]:
pip install streamlit pandas scikit-learn

In [35]:
import pickle

# On sauvegarde les 3 éléments clés de VOTRE modèle
# 1. Le DataFrame avec les titres
pickle.dump(df, open('movies.pkl', 'wb'))

# 2. La matrice des features (X) que vous avez créée cellule 11
pickle.dump(X, open('X_matrix.pkl', 'wb'))

# 3. Le modèle KNN entraîné cellule 12
pickle.dump(model_knn, open('knn_model.pkl', 'wb'))

print("Fichiers sauvegardés avec succès : movies.pkl, X_matrix.pkl, knn_model.pkl")

Fichiers sauvegardés avec succès : movies.pkl, X_matrix.pkl, knn_model.pkl
